# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **FPL player score** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, combine predictions with static data
3. Store predictions in a monitoring feature group

## <span style='color:#ff5f27'> 📝 Imports

In [299]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from functions import util
import os
from importlib import reload

In [ ]:
reload(util)

print(util.get_gameweek_from_date("2025-1-4"))

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)

# Get the gameweek from the date
gameweek = util.get_gameweek_from_date(str(today.date()))
last_gameweek = gameweek - 1

gameweek
last_gameweek

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [ ]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
fs = project.get_feature_store()

## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="player_score_xgboost_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

## <span style="color:#ff5f27;">✨ Get Player Score Features From Previous Gameweek </span>



In [ ]:
players_fg = fs.get_feature_group(
    name='player_features',
    version=1,
)
input_data = players_fg.filter(players_fg.gameweek == last_gameweek).read()
output_data = players_fg.filter(players_fg.gameweek == gameweek).read()

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [ ]:
inputs = ['prev_minutes', 'prev_goals_scored', 'prev_assists',
         'prev_clean_sheets', 'prev_goals_conceded', 'prev_own_goals',
         'prev_penalties_saved', 'prev_penalties_missed', 'prev_yellow_cards',
         'prev_red_cards', 'prev_saves', 'prev_bonus',
         'prev_bps', 'prev_influence', 'prev_creativity',
         'prev_threat', 'prev_ict_index', 'prev_starts',
         'prev_expected_goals', 'prev_expected_assists', 'prev_expected_goal_involvements',
         'prev_expected_goals_conceded', 'prev_in_dreamteam']

for i in range(0, len(input_data)):
    output_data.at[i, 'predicted_score'] = retrieved_xgboost_model.predict(input_data.loc[i, inputs].values.reshape(1, -1))[0]

output_data

In [ ]:
output_data.info()

### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

### Create Score Graph
Draw a graph of the predictions with all the players as a PNG and save it to the github

In [ ]:
file_path = "./player_score_model/images/predicted_scores.png"
plt = util.plot_player_score_forecast("FPL Player scores", output_data, file_path)
plt.show()

In [309]:
# Get or create feature group
monitor_fg = fs.create_feature_group(
    name='fpl_predictions',
    description='FPL player score prediction monitoring',
    version=1,
    primary_key=['id', 'gameweek']
)

In [ ]:
# Only keep the relevant columns
output_data = output_data[['id', 'total_points', 'gameweek', 'predicted_score']]

output_data

In [ ]:
# Add static data
for i in range(0, len(output_data)):
    player_id = output_data.loc[i, 'id']

    player_data = util.get_player_info(player_id)
    for key in player_data:
        output_data.at[i, key] = player_data[key]

In [ ]:
output_data.head()

# Add empty column
output_data['previous_predicted_score'] = 0

In [ ]:
# Get previous predictions if present
previous_predictions = []

# Check if minotoring feature group is empty by checking if it can be read without errors
try:
    monitor_fg.read()
except:
    skip = True

if skip == False:
    latest_gameweeks = monitor_fg.filter(monitor_fg.gameweek >= (int(gameweek) - 5) and monitor_fg.gameweek <= int(gameweek)).read()

    latest_gameweeks

    for i in range(0, len(output_data)):
        player_id = output_data.loc[i, 'id']

        # Check if the player has been predicted before
        player_predictions = latest_gameweeks[latest_gameweeks.id == player_id]


        if len(player_predictions) > 0:
            # Sort the predictions by gameweek
            player_predictions = player_predictions.sort_values(by='gameweek', ascending=False)

            # only keep the relevant columns
            player_predictions = player_predictions[['total_points', 'gameweek', 'predicted_score']]

            # print(player_predictions.info())

            # Update the output data with the previous prediction
            output_data.at[i, 'previous_predicted_score'] = player_predictions.to_dict('records')
        else:
            pass

    output_data
    # output_data_copy = pd.DataFrame(output_data_copy)

    # output_data_copy = output_data_copy.dropna(subset=['previous_predicted_score'])
    # print(output_data_copy)
else:
    print("No previous data found in feature group")

In [ ]:
print(output_data)

In [ ]:
monitor_fg.insert(output_data)

In [ ]:
monitor_fg.update_feature_description("id", "Player ID")
monitor_fg.update_feature_description("total_points", "Total points of the player in the gameweek (label)")
monitor_fg.update_feature_description("gameweek", "Gameweek")
monitor_fg.update_feature_description("predicted_score", "Predicted score of the player in the gameweek")
monitor_fg.update_feature_description("name", "Player name")
monitor_fg.update_feature_description("team", "Player team")
monitor_fg.update_feature_description("position", "Player position")
monitor_fg.update_feature_description("previous_predicted_score", "Previous (max 5) predicted scores of the player")


---